In [1]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [2]:
import pandas as pd
crop = pd.read_csv('/content/drive/MyDrive/crop_recommendation.csv')
X = crop.iloc[:,:-1].values
Y = crop.iloc[:,-1].values

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn import model_selection

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.30)

In [5]:
models = []
models.append(('SVC', SVC(gamma ='auto', probability = True)))
models.append(('svm1', SVC(probability=True, kernel='poly', degree=1)))
models.append(('svm2', SVC(probability=True, kernel='poly', degree=2)))
models.append(('svm3', SVC(probability=True, kernel='poly', degree=3)))
models.append(('svm4', SVC(probability=True, kernel='poly', degree=4)))
models.append(('svm5', SVC(probability=True, kernel='poly', degree=5)))
models.append(('rf',RandomForestClassifier(n_estimators = 21)))
models.append(('gnb',GaussianNB()))
models.append(('knn1', KNeighborsClassifier(n_neighbors=1)))
models.append(('knn3', KNeighborsClassifier(n_neighbors=3)))
models.append(('knn5', KNeighborsClassifier(n_neighbors=5)))
models.append(('knn7', KNeighborsClassifier(n_neighbors=7)))
models.append(('knn9', KNeighborsClassifier(n_neighbors=9)))

In [6]:
vot_soft = VotingClassifier(estimators=models, voting='soft')
vot_soft.fit(X_train, y_train)
y_pred = vot_soft.predict(X_test)

In [7]:
scores = model_selection.cross_val_score(vot_soft, X_test, y_test,cv=5,scoring='accuracy')
print("Accuracy: ",scores.mean())

Accuracy:  0.9863636363636363


In [8]:
score = accuracy_score(y_test, y_pred)
print("Voting Score % d" % score)

Voting Score  0


### CNN model


In [9]:
#importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense, Dropout
from keras import optimizers

In [10]:
# Initialing the CNN
classifier = Sequential()

# Step 1 - Convolution Layer
classifier.add(Convolution2D(32, 3,  3, input_shape = (64, 64, 3), activation = 'relu'))

#step 2 - Pooling
classifier.add(MaxPooling2D(pool_size =(2,2)))

# Adding second convolution layer
classifier.add(Convolution2D(32, 3,  3, activation = 'relu'))
classifier.add(MaxPooling2D(pool_size =(2,2)))


#Step 3 - Flattening
classifier.add(Flatten())

#Step 4 - Full Connection
classifier.add(Dense(256, activation = 'relu'))
classifier.add(Dropout(0.5))
classifier.add(Dense(10, activation = 'softmax'))

In [11]:
#Compiling The CNN
classifier.compile(
              optimizer = 'adam',
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'])

In [12]:
#Part 2 Fittting the CNN to the image
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

In [15]:
test_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory(
        '/content/drive/MyDrive/train',
        target_size=(64, 64),
        batch_size=32,
        class_mode='categorical')

test_set = test_datagen.flow_from_directory(
        '/content/drive/MyDrive/test',
        target_size=(64, 64),
        batch_size=32,
        class_mode='categorical')

model = classifier.fit(
        training_set,
        steps_per_epoch=50,
        epochs=100,
        validation_data = test_set,
        validation_steps = 6500
      )

Found 3001 images belonging to 10 classes.
Found 500 images belonging to 10 classes.
Epoch 1/100
50/50 [==============================] - 11s 220ms/step - loss: 1.3515 - accuracy: 0.5235 - val_loss: 1.3157 - val_accuracy: 0.5260
Epoch 2/100
50/50 [==============================] - 8s 162ms/step - loss: 1.3635 - accuracy: 0.5163
Epoch 3/100
50/50 [==============================] - 8s 162ms/step - loss: 1.3182 - accuracy: 0.5323
Epoch 4/100
50/50 [==============================] - 8s 166ms/step - loss: 1.3419 - accuracy: 0.5381
Epoch 5/100
50/50 [==============================] - 8s 163ms/step - loss: 1.3489 - accuracy: 0.5304
Epoch 6/100
50/50 [==============================] - 8s 154ms/step - loss: 1.3384 - accuracy: 0.5231
Epoch 7/100
50/50 [==============================] - 8s 162ms/step - loss: 1.3197 - accuracy: 0.5468
Epoch 8/100
50/50 [==============================] - 8s 166ms/step - loss: 1.2775 - accuracy: 0.5337
Epoch 9/100
50/50 [==============================] - 8s 160ms/st

## Save the model

In [16]:
#Saving the model
import h5py
classifier.save('Trained_Model.h5')

In [17]:
import pickle as pkl
filename = 'Pest.sav'
pkl.dump(model, open(filename, 'wb'))

INFO:tensorflow:Assets written to: ram://cbae7d0f-8471-46a0-a420-68da15e0e5f5/assets
